In [0]:
# cost of misclassification is high 
# p of pair of questions , setting thresold
# no strict latency concerns
# interpretability is partially important 

# **Type of machine learning problem--- binary classification**

# **Performance metrics--- logloss**

# **Exploratory Data Analysis**